In [1]:
!pip3 install wordcloud-fa
!pip3 install bokeh
!pip3 install pyldavis
!pip3 install git+https://github.com/rwalk/gsdmm.git

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/rwalk/gsdmm.git to /tmp/pip-req-build-prkppzkq
  Running command git clone --filter=blob:none --quiet https://github.com/rwalk/gsdmm.git /tmp/pip-req-build-prkppzkq
  Resolved https://github.com/rwalk/gsdmm.git to commit 4ad1b6b6976743681ee4976b4573463d359214ee
  Preparing metadata (setup.py) ... done


In [1]:
import warnings
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from matplotlib.pyplot import figure
import ast
import gensim
from gensim import corpora, models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors
from wordcloud_fa import WordCloudFa
from matplotlib.patches import Rectangle
from matplotlib.ticker import FuncFormatter
import time
import random
import math

from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

# python 3.6
# import pickle5 as pickle
# python 3.8
import pickle

from gsdmm import MovieGroupProcess
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
base_path = "../data/"

In [3]:
input_file = base_path + 'preprocessed.csv'
tqdm.pandas()
df = pd.read_csv(input_file)
df

/tmp/ipykernel_32133/250796820.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


,conversation_id,retweets_count,mentions,created_at,likes_count,in_reply_to_user_id,_id,source,user_id,in_reply_to_status_id,...,quoted_status_id,cashtags,urls,tweet,hashtags,tweet_tokens,lemmatized_tweet,lemmatized_tweet_token,tweet_size,lemmatized_tweet_size
0,1375557622410149894,2,NaN,2021-03-26 21:23:21+00:00,111,5.720514e+08,1375559020451356674,Twitter for iPhone,572051429,1.375558e+18,...,NaN,NaN,NaN,گزارش حقوقش میدونم سالی ۸۵ همسرش دانشجوخودش دک...,[],"['گزارش', 'حقوقش', 'میدونم', 'سالی', '۸۵', 'هم...",گزارش حقوق میدونم ۸۵ همسر دانشجوخودش دکترا شرک...,"['گزارش', 'حقوق', 'میدونم', '۸۵', 'همسر', 'دان...",32,30
1,1375559420176908296,3,NaN,2021-03-26 21:24:57+00:00,163,NaN,1375559420176908296,Twitter for iPhone,812764546012246017,NaN,...,NaN,NaN,NaN,اینایی سفر مهمونی پروتکلی رعایت نکردن سرکار خط...,[],"['اینایی', 'سفر', 'مهمونی', 'پروتکلی', 'رعایت'...",اینایی سفر مهمونی پروتکل رعایت نکردن سرکار خطر...,"['اینایی', 'سفر', 'مهمونی', 'پروتکل', 'رعایت',...",11,11
2,1383828354197979136,2,NaN,2021-04-18 17:02:44+00:00,37,NaN,1383828354197979136,Twitter for Android,985101690536898565,NaN,...,NaN,NaN,NaN,خسته دورکاری داشتمتو شهرهای سروصدا شب ازصبح آز...,[],"['خسته', 'دورکاری', 'داشتمتو', 'شهرهای', 'سروص...",خسته دورکاری داشتمتو شهر سروصدا شب ازصبح آزارد...,"['خسته', 'دورکاری', 'داشتمتو', 'شهر', 'سروصدا'...",27,26
3,1383840315031781385,0,NaN,2021-04-18 17:50:16+00:00,11,NaN,1383840315031781385,Twitter Web App,310871673,NaN,...,1.383657e+18,NaN,NaN,آبان ۹۸ دورکار,[],"['آبان', '۹۸', 'دورکار']",آبان ۹۸ دورکار,"['آبان', '۹۸', 'دورکار']",3,3
4,1383820625354002439,1,['15782990'],2021-04-18 17:31:42+00:00,9,1.578299e+07,1383835643239747593,Twitter Web App,1271095835028307969,1.383821e+18,...,NaN,NaN,NaN,حتمن میرفتم شهری اقوام دوستام نزدیکتر خونه اجا...,[],"['حتمن', 'میرفتم', 'شهری', 'اقوام', 'دوستام', ...",حتمن میرفتم شهر اقوام دوستام خونه اجازه میکردم...,"['حتمن', 'میرفتم', 'شهر', 'اقوام', 'دوستام', '...",16,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45354,1434127275587428352,0,['1273546506012897281'],2021-09-04 12:46:01+00:00,3,1.273547e+18,1434135662396018689,Twitter for Android,1260775960267444225,1.434127e+18,...,NaN,NaN,NaN,لجم میگیره کثافت دورکاری سخت کار اداره ست ذهنا...,[],"['لجم', 'میگیره', 'کثافت', 'دورکاری', 'سخت', '...",لج میگیره کثافت دورکاری سخت کار اداره ست ذهنای...,"['لج', 'میگیره', 'کثافت', 'دورکاری', 'سخت', 'ک...",15,15
45355,1434127961943326723,0,['1251428296589488129'],2021-09-04 12:21:54+00:00,2,1.251428e+18,1434129593036939267,Twitter for Android,1214804130096668672,1.434128e+18,...,NaN,NaN,NaN,اطلاع ماهه رزومه فرستادم جواب ندادن آگهی‌هاشون...,[],"['اطلاع', 'ماهه', 'رزومه', 'فرستادم', 'جواب', ...",اطلاع ماهه رزومه فرستاد جواب ندادن آگهی‌هاشون ...,"['اطلاع', 'ماهه', 'رزومه', 'فرستاد', 'جواب', '...",9,9
45356,1434138057071112194,0,NaN,2021-09-04 12:55:32+00:00,4,NaN,1434138057071112194,Twitter for iPhone,1147481863138811905,NaN,...,NaN,NaN,NaN,خواهرم کار استخدام بشه دورکاری الآن نخواهم_تون...,[],"['خواهرم', 'کار', 'استخدام', 'بشه', 'دورکاری',...",خواهر کار استخدام بشه دورکاری الآن نخواهم_تونس...,"['خواهر', 'کار', 'استخدام', 'بشه', 'دورکاری', ...",15,15
45357,1434133064750338048,0,NaN,2021-09-04 12:35:42+00:00,18,NaN,1434133064750338048,Twitter for Android,1106832761216815104,NaN,...,NaN,NaN,NaN,دورکاری,[],['دورکاری'],دورکاری,['دورکاری'],1,1


In [4]:
token_list_lemm = []
token_list = []
for ind in df.index:
    # print(type(df['lemmatized_tweet_token'][ind]))
    token_list_lemm += [ast.literal_eval(df['lemmatized_tweet_token'][ind])]
    token_list += [ast.literal_eval(df['tweet_tokens'][ind])]

In [5]:
def get_dictionary(tokens):
    dictionary = corpora.Dictionary(tokens)
    dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)    
    return dictionary

In [6]:
def get_topics_lists(model, top_clusters, n_words, dictionary):
    topics = []
    
    # iterate over top n clusters
    for cluster in top_clusters:
        #create sorted dictionary of word distributions
        sorted_dict = sorted(model.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:n_words]
         
        #create empty list to contain words
        topic = []
        
        #iterate over top n words in topic
        for k,v in sorted_dict:
            if (k in dictionary.token2id): 
                #append words to topic list
                topic.append(k)
        #append topics to topics list    
        topics.append(topic)
    
    return topics

In [7]:
def show_gsdm_results(token_list, dictionary, corpus, topic_range = [6, 8, 9, 10], iter_range = [10, 15, 20, 30]):
    vocab_length = len(dictionary)
    info_list = []
    for t in topic_range:
        for i in iter_range:
            gsdmm = MovieGroupProcess(K=t, alpha=0.01, beta=0.01, n_iters=i)
            y = gsdmm.fit(tqdm(token_list), vocab_length)
            doc_count = np.array(gsdmm.cluster_doc_count)
            
            
            # Topics sorted by the number of document they are allocated to
            top_index = doc_count.argsort()[-15:][::-1]
            print("top_index: ", top_index)

            # get topics to feed to coherence model
            topics = get_topics_lists(gsdmm, top_index, 10, dictionary)
            
            # evaluate model using Topic Coherence score
            cm_gsdmm = CoherenceModel(topics=topics, 
                          dictionary=dictionary, 
                          corpus=corpus, 
                          texts=token_list, 
                          coherence='c_v')
            
            # get coherence value
            coherence_gsdmm = cm_gsdmm.get_coherence()
            
            info_list.append({'topics': t, 'iterations': i, 'docs_per_topic': doc_count, 'coherence': coherence_gsdmm})
    results = pd.DataFrame(info_list).sort_values(by='coherence', ascending=False)
    display(results)
    return info_list

In [8]:
def get_bow_corpus(dictionary, tokens):
    return [dictionary.doc2bow(doc) for doc in tokens]

In [9]:
def check_lem_in_gsdm():
    dictionary_lemm = get_dictionary(token_list_lemm)
    dictionary = get_dictionary(token_list)
    
    corpus_lemm = get_bow_corpus(dictionary_lemm, token_list_lemm)
    corpus = get_bow_corpus(dictionary, token_list)
    
    print("With Lemmatization")
    show_gsdm_results(token_list_lemm, dictionary_lemm, corpus_lemm, [3, 6], [3, 6])
    print("Without Lemmatization")
    show_gsdm_results(token_list, dictionary, corpus, [3, 6], [3, 6])
    
check_lem_in_gsdm()

With Lemmatization


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 29184.80it/s]


In stage 0: transferred 26789 clusters with 3 clusters populated
In stage 1: transferred 18317 clusters with 3 clusters populated
In stage 2: transferred 11716 clusters with 3 clusters populated
top_index:  [1 2 0]


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 29188.34it/s]


In stage 0: transferred 26365 clusters with 3 clusters populated
In stage 1: transferred 17796 clusters with 3 clusters populated
In stage 2: transferred 11524 clusters with 3 clusters populated
In stage 3: transferred 6568 clusters with 3 clusters populated
In stage 4: transferred 4102 clusters with 3 clusters populated
In stage 5: transferred 3303 clusters with 3 clusters populated
top_index:  [1 2 0]


...

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 52808.02it/s]


In stage 0: transferred 33027 clusters with 6 clusters populated
In stage 1: transferred 22469 clusters with 6 clusters populated
In stage 2: transferred 16160 clusters with 6 clusters populated
top_index:  [1 0 5 2 4 3]


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 39133.72it/s]


In stage 0: transferred 32966 clusters with 6 clusters populated
In stage 1: transferred 22195 clusters with 6 clusters populated
In stage 2: transferred 16928 clusters with 6 clusters populated
In stage 3: transferred 12095 clusters with 6 clusters populated
In stage 4: transferred 9626 clusters with 6 clusters populated
In stage 5: transferred 8556 clusters with 6 clusters populated
top_index:  [4 2 1 5 3 0]


,topics,iterations,docs_per_topic,coherence
3,6,6,"[1675, 8090, 10400, 3718, 15951, 5525]",0.534011
1,3,6,"[22059, 19563, 3737]",0.526466
2,6,3,"[9741, 13499, 7256, 2179, 4931, 7753]",0.499383
0,3,3,"[22194, 7330, 15835]",0.419168


## TF-IDF

In [10]:
def get_tf_idf_corpus(bow_corpus):
    tfidf = models.TfidfModel(bow_corpus)
    return tfidf[bow_corpus]

In [11]:
def check_gsdm_tf_idf():
    dictionary = get_dictionary(token_list_lemm)
    bow_corpus = get_bow_corpus(dictionary, token_list_lemm)
    tf_idf_corpus = get_tf_idf_corpus(bow_corpus)
    show_gsdm_results(token_list_lemm, dictionary, tf_idf_corpus, [6], [3, 6])

check_gsdm_tf_idf()

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 44609.38it/s]


In stage 0: transferred 33569 clusters with 6 clusters populated
In stage 1: transferred 23232 clusters with 6 clusters populated
In stage 2: transferred 16589 clusters with 6 clusters populated
top_index:  [0 1 4 2 5 3]


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 36896.50it/s]


In stage 0: transferred 33692 clusters with 6 clusters populated
In stage 1: transferred 23399 clusters with 6 clusters populated
In stage 2: transferred 15891 clusters with 6 clusters populated
In stage 3: transferred 11632 clusters with 6 clusters populated
In stage 4: transferred 9979 clusters with 6 clusters populated
In stage 5: transferred 9168 clusters with 6 clusters populated
top_index:  [1 3 5 0 4 2]


,topics,iterations,docs_per_topic,coherence
0,6,3,"[13250, 8983, 7397, 2076, 8566, 5087]",0.520986
1,6,6,"[5899, 13957, 1817, 10311, 4859, 8516]",0.516272


In [12]:
dictionary_lemm = get_dictionary(token_list_lemm)
corpus_lemm = get_bow_corpus(dictionary_lemm, token_list_lemm)

start = time.time()
gsdm_results_100 = show_gsdm_results(token_list_lemm, dictionary_lemm, corpus_lemm)
end = time.time()

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 39826.24it/s]


In stage 0: transferred 33487 clusters with 6 clusters populated
In stage 1: transferred 23195 clusters with 6 clusters populated
In stage 2: transferred 15889 clusters with 6 clusters populated
In stage 3: transferred 11478 clusters with 6 clusters populated
In stage 4: transferred 9581 clusters with 6 clusters populated
In stage 5: transferred 8684 clusters with 6 clusters populated
In stage 6: transferred 8262 clusters with 6 clusters populated
In stage 7: transferred 8019 clusters with 6 clusters populated
In stage 8: transferred 7815 clusters with 6 clusters populated
In stage 9: transferred 7511 clusters with 6 clusters populated
top_index:  [2 1 4 5 0 3]


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 48808.85it/s]


In stage 0: transferred 33474 clusters with 6 clusters populated
In stage 1: transferred 22071 clusters with 6 clusters populated
In stage 2: transferred 17084 clusters with 6 clusters populated
In stage 3: transferred 13976 clusters with 6 clusters populated
In stage 4: transferred 11779 clusters with 6 clusters populated
In stage 5: transferred 10510 clusters with 6 clusters populated
In stage 6: transferred 9388 clusters with 6 clusters populated
In stage 7: transferred 8665 clusters with 6 clusters populated
In stage 8: transferred 8233 clusters with 6 clusters populated
In stage 9: transferred 7809 clusters with 6 clusters populated
In stage 10: transferred 7566 clusters with 6 clusters populated
In stage 11: transferred 7519 clusters with 6 clusters populated
In stage 12: transferred 7307 clusters with 6 clusters populated
In stage 13: transferred 7159 clusters with 6 clusters populated
In stage 14: transferred 7028 clusters with 6 clusters populated
top_index:  [1 2 5 0 3 4]


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 45593.15it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 46727.61it/s]


In stage 0: transferred 36585 clusters with 10 clusters populated
In stage 1: transferred 25162 clusters with 10 clusters populated
In stage 2: transferred 18483 clusters with 10 clusters populated
In stage 3: transferred 14687 clusters with 10 clusters populated
In stage 4: transferred 12670 clusters with 10 clusters populated
In stage 5: transferred 11582 clusters with 10 clusters populated
In stage 6: transferred 10834 clusters with 10 clusters populated
In stage 7: transferred 10533 clusters with 10 clusters populated
In stage 8: transferred 10135 clusters with 10 clusters populated
In stage 9: transferred 9857 clusters with 10 clusters populated
In stage 10: transferred 9696 clusters with 10 clusters populated
In stage 11: transferred 9708 clusters with 10 clusters populated
In stage 12: transferred 9467 clusters with 10 clusters populated
In stage 13: transferred 9325 clusters with 10 clusters populated
In stage 14: transferred 9160 clusters with 10 clusters populated
In stage 15

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 38449.34it/s]


In stage 0: transferred 36700 clusters with 10 clusters populated
In stage 1: transferred 24409 clusters with 10 clusters populated
In stage 2: transferred 16849 clusters with 10 clusters populated
In stage 3: transferred 13986 clusters with 10 clusters populated
In stage 4: transferred 12811 clusters with 10 clusters populated
In stage 5: transferred 11816 clusters with 10 clusters populated
In stage 6: transferred 11231 clusters with 10 clusters populated
In stage 7: transferred 10904 clusters with 10 clusters populated
In stage 8: transferred 10560 clusters with 10 clusters populated
In stage 9: transferred 10182 clusters with 10 clusters populated
In stage 10: transferred 9904 clusters with 10 clusters populated
In stage 11: transferred 9909 clusters with 10 clusters populated
In stage 12: transferred 9530 clusters with 10 clusters populated
In stage 13: transferred 9374 clusters with 10 clusters populated
In stage 14: transferred 9414 clusters with 10 clusters populated
In stage 1

,topics,iterations,docs_per_topic,coherence
0,6,10,"[1973, 14197, 14297, 1132, 7184, 6576]",0.587102
15,10,30,"[4479, 10668, 2906, 8565, 1301, 2521, 5692, 60...",0.572663
11,9,30,"[5060, 4577, 3124, 10107, 5677, 1049, 13090, 1...",0.567221
6,8,20,"[3896, 9314, 1489, 1242, 12380, 3002, 7322, 6714]",0.560331
14,10,20,"[11423, 1524, 7000, 2175, 5022, 5776, 4478, 12...",0.559105
3,6,30,"[9987, 1318, 4592, 3405, 9329, 16728]",0.559093
2,6,20,"[9743, 7615, 4036, 2749, 1825, 19391]",0.556768
4,8,10,"[4027, 1519, 5500, 1213, 7589, 11204, 10321, 3...",0.555131
12,10,10,"[5693, 4867, 9273, 1961, 6273, 3907, 5951, 483...",0.555073
5,8,15,"[11474, 1509, 1756, 5577, 4707, 7026, 3891, 9419]",0.550751


In [13]:
def show_gsdm_with_hyper_params(best_models, token_list, dictionary, corpus, alpha_range = [0.01, 0.1], beta_range = [0.01, 0.1], top_words_range = [10, 20]):
    vocab_length = len(dictionary)
    info_list = []
    for model in best_models:
        for alpha in alpha_range:
            for beta in beta_range:
                gsdmm = MovieGroupProcess(K=model['topics'], alpha=alpha, beta=beta, n_iters=model['iterations'])
                y = gsdmm.fit(tqdm(token_list), vocab_length)
                doc_count = np.array(gsdmm.cluster_doc_count)

                top_index = doc_count.argsort()[-15:][::-1]

                coherences = []
                info = {'topics': model['topics'], 'iterations': model['iterations'], 'alpha': alpha, 'beta': beta, 'docs_per_topic': doc_count}
                for words in top_words_range:
                    topics = get_topics_lists(gsdmm, top_index, words, dictionary)

                    cm_gsdmm = CoherenceModel(topics=topics, 
                                  dictionary=dictionary, 
                                  corpus=corpus, 
                                  texts=token_list, 
                                  coherence='c_v')

                    info['topic_' + str(words)] = cm_gsdmm.get_coherence()

                info_list.append(info)
    results = pd.DataFrame(info_list)
    display(results)
    return info_list

In [14]:
def compare_gsdm_hyper_params():
        best_gsdm_models = [{'topics': 9, 'iterations': 10},
                   {'topics': 10, 'iterations': 15},
                   {'topics': 8, 'iterations': 20}]
        dictionary_lemm = get_dictionary(token_list_lemm)
        corpus_lemm = get_bow_corpus(dictionary_lemm, token_list_lemm)

        return show_gsdm_with_hyper_params(best_gsdm_models, token_list_lemm, dictionary_lemm, corpus_lemm) 

gsdm_results = compare_gsdm_hyper_params()

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 44020.61it/s]


In stage 0: transferred 35995 clusters with 9 clusters populated
In stage 1: transferred 24150 clusters with 9 clusters populated
In stage 2: transferred 17738 clusters with 9 clusters populated
In stage 3: transferred 15063 clusters with 9 clusters populated
In stage 4: transferred 13567 clusters with 9 clusters populated
In stage 5: transferred 12519 clusters with 9 clusters populated
In stage 6: transferred 12159 clusters with 9 clusters populated
In stage 7: transferred 11907 clusters with 9 clusters populated
In stage 8: transferred 11533 clusters with 9 clusters populated
In stage 9: transferred 11080 clusters with 9 clusters populated


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 42024.22it/s]


In stage 0: transferred 37170 clusters with 9 clusters populated
In stage 1: transferred 26801 clusters with 9 clusters populated
In stage 2: transferred 18259 clusters with 9 clusters populated
In stage 3: transferred 14644 clusters with 9 clusters populated
In stage 4: transferred 12930 clusters with 9 clusters populated
In stage 5: transferred 12120 clusters with 9 clusters populated
In stage 6: transferred 11423 clusters with 9 clusters populated
In stage 7: transferred 10837 clusters with 9 clusters populated
In stage 8: transferred 10441 clusters with 9 clusters populated
In stage 9: transferred 10350 clusters with 9 clusters populated


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 44524.85it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 37011.35it/s]


In stage 0: transferred 35396 clusters with 8 clusters populated
In stage 1: transferred 24214 clusters with 8 clusters populated
In stage 2: transferred 17307 clusters with 8 clusters populated
In stage 3: transferred 14115 clusters with 8 clusters populated
In stage 4: transferred 12297 clusters with 8 clusters populated
In stage 5: transferred 11291 clusters with 8 clusters populated
In stage 6: transferred 10684 clusters with 8 clusters populated
In stage 7: transferred 10190 clusters with 8 clusters populated
In stage 8: transferred 9747 clusters with 8 clusters populated
In stage 9: transferred 9506 clusters with 8 clusters populated
In stage 10: transferred 9357 clusters with 8 clusters populated
In stage 11: transferred 9162 clusters with 8 clusters populated
In stage 12: transferred 8952 clusters with 8 clusters populated
In stage 13: transferred 8909 clusters with 8 clusters populated
In stage 14: transferred 8752 clusters with 8 clusters populated
In stage 15: transferred 88

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 40195.95it/s]


In stage 0: transferred 36338 clusters with 8 clusters populated
In stage 1: transferred 26652 clusters with 8 clusters populated
In stage 2: transferred 19273 clusters with 8 clusters populated
In stage 3: transferred 14789 clusters with 8 clusters populated
In stage 4: transferred 12921 clusters with 8 clusters populated
In stage 5: transferred 11805 clusters with 8 clusters populated
In stage 6: transferred 11129 clusters with 8 clusters populated
In stage 7: transferred 10865 clusters with 8 clusters populated
In stage 8: transferred 10462 clusters with 8 clusters populated
In stage 9: transferred 10311 clusters with 8 clusters populated
In stage 10: transferred 10231 clusters with 8 clusters populated
In stage 11: transferred 10014 clusters with 8 clusters populated
In stage 12: transferred 9914 clusters with 8 clusters populated
In stage 13: transferred 9940 clusters with 8 clusters populated
In stage 14: transferred 9803 clusters with 8 clusters populated
In stage 15: transferre

,topics,iterations,alpha,beta,docs_per_topic,topic_10,topic_20
0,9,10,0.01,0.01,"[7530, 4806, 4066, 6588, 1468, 8233, 5701, 314...",0.510650,0.479391
1,9,10,0.01,0.10,"[5496, 10027, 1666, 3198, 1045, 9360, 4731, 58...",0.565161,0.552191
2,9,10,0.10,0.01,"[1353, 6195, 5047, 1118, 5263, 5885, 5457, 903...",0.547446,0.519961
3,9,10,0.10,0.10,"[3741, 5616, 10203, 3768, 3692, 1345, 6885, 34...",0.528139,0.496500
4,10,15,0.01,0.01,"[4183, 4635, 3931, 7162, 7231, 3795, 6831, 142...",0.545285,0.491894
5,10,15,0.01,0.10,"[3909, 3222, 5106, 4566, 2037, 8505, 4651, 809...",0.550649,0.509035
6,10,15,0.10,0.01,"[5886, 3695, 9768, 6005, 3992, 2054, 2045, 588...",0.548703,0.523781
7,10,15,0.10,0.10,"[7834, 3121, 985, 6875, 1603, 4076, 2999, 8053...",0.586275,0.554079
8,8,20,0.01,0.01,"[2827, 7009, 1629, 9441, 10403, 4182, 6418, 3450]",0.548824,0.516253
9,8,20,0.01,0.10,"[1027, 5418, 7658, 3281, 8636, 5811, 11690, 1838]",0.563315,0.573174


In [15]:
def compare_gsdm_hyper_params():
        best_gsdm_models = [{'topics': 3, 'iterations': 10},
                   {'topics': 6, 'iterations': 15},
                   {'topics': 8, 'iterations': 20}]
        dictionary_lemm = get_dictionary(token_list_lemm)
        corpus_lemm = get_bow_corpus(dictionary_lemm, token_list_lemm)

        return show_gsdm_with_hyper_params(best_gsdm_models, token_list_lemm, dictionary_lemm, corpus_lemm) 

gsdm_results = compare_gsdm_hyper_params()

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 48611.12it/s]


In stage 0: transferred 26584 clusters with 3 clusters populated
In stage 1: transferred 17507 clusters with 3 clusters populated
In stage 2: transferred 12497 clusters with 3 clusters populated
In stage 3: transferred 9209 clusters with 3 clusters populated
In stage 4: transferred 6956 clusters with 3 clusters populated
In stage 5: transferred 5166 clusters with 3 clusters populated
In stage 6: transferred 3736 clusters with 3 clusters populated
In stage 7: transferred 2764 clusters with 3 clusters populated
In stage 8: transferred 2270 clusters with 3 clusters populated
In stage 9: transferred 2059 clusters with 3 clusters populated


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 54285.42it/s]


In stage 0: transferred 27146 clusters with 3 clusters populated
In stage 1: transferred 20139 clusters with 3 clusters populated
In stage 2: transferred 12791 clusters with 3 clusters populated
In stage 3: transferred 7381 clusters with 3 clusters populated
In stage 4: transferred 5229 clusters with 3 clusters populated
In stage 5: transferred 4604 clusters with 3 clusters populated
In stage 6: transferred 4385 clusters with 3 clusters populated
In stage 7: transferred 4231 clusters with 3 clusters populated
In stage 8: transferred 4176 clusters with 3 clusters populated
In stage 9: transferred 4089 clusters with 3 clusters populated


100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 59642.57it/s]


...

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:01<00:00, 40285.83it/s]


In stage 0: transferred 35587 clusters with 8 clusters populated
In stage 1: transferred 24157 clusters with 8 clusters populated
In stage 2: transferred 16802 clusters with 8 clusters populated
In stage 3: transferred 13215 clusters with 8 clusters populated
In stage 4: transferred 11639 clusters with 8 clusters populated
In stage 5: transferred 10490 clusters with 8 clusters populated
In stage 6: transferred 9861 clusters with 8 clusters populated
In stage 7: transferred 9494 clusters with 8 clusters populated
In stage 8: transferred 9094 clusters with 8 clusters populated
In stage 9: transferred 8755 clusters with 8 clusters populated
In stage 10: transferred 8489 clusters with 8 clusters populated
In stage 11: transferred 8271 clusters with 8 clusters populated
In stage 12: transferred 8126 clusters with 8 clusters populated
In stage 13: transferred 7886 clusters with 8 clusters populated
In stage 14: transferred 7757 clusters with 8 clusters populated
In stage 15: transferred 7640

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 66186.47it/s]


In stage 0: transferred 36765 clusters with 8 clusters populated
In stage 1: transferred 25460 clusters with 8 clusters populated
In stage 2: transferred 17074 clusters with 8 clusters populated
In stage 3: transferred 13945 clusters with 8 clusters populated
In stage 4: transferred 12659 clusters with 8 clusters populated
In stage 5: transferred 11721 clusters with 8 clusters populated
In stage 6: transferred 11143 clusters with 8 clusters populated
In stage 7: transferred 10671 clusters with 8 clusters populated
In stage 8: transferred 10200 clusters with 8 clusters populated
In stage 9: transferred 10113 clusters with 8 clusters populated
In stage 10: transferred 9777 clusters with 8 clusters populated
In stage 11: transferred 9617 clusters with 8 clusters populated
In stage 12: transferred 9381 clusters with 8 clusters populated
In stage 13: transferred 9197 clusters with 8 clusters populated
In stage 14: transferred 9184 clusters with 8 clusters populated
In stage 15: transferred 

,topics,iterations,alpha,beta,docs_per_topic,topic_10,topic_20
0,3,10,0.01,0.01,"[6796, 2534, 36029]",0.621839,0.586884
1,3,10,0.01,0.10,"[11648, 26769, 6942]",0.426853,0.345194
2,3,10,0.10,0.01,"[7001, 9944, 28414]",0.462967,0.443506
3,3,10,0.10,0.10,"[6991, 11510, 26858]",0.426853,0.342123
4,6,15,0.01,0.01,"[7062, 2178, 17864, 4281, 8099, 5875]",0.441506,0.433981
5,6,15,0.01,0.10,"[17047, 4990, 7855, 2051, 8724, 4692]",0.554627,0.526189
6,6,15,0.10,0.01,"[2658, 12177, 1776, 8409, 5283, 15056]",0.557303,0.532858
7,6,15,0.10,0.10,"[7017, 1630, 10139, 3715, 17350, 5508]",0.545892,0.537049
8,8,20,0.01,0.01,"[15612, 2060, 968, 5097, 8236, 6933, 4961, 1492]",0.594920,0.585331
9,8,20,0.01,0.10,"[4529, 3559, 3248, 1632, 8030, 12389, 3705, 8267]",0.560473,0.536433


In [16]:
def get_data_frame(chunk_size = 10 ** 5, is_chunked = True):
    input_file = base_path + 'prep1_tweets.csv'
    file_size = 38 * 10 ** 5

    tqdm.pandas()

    reader = pd.read_csv(input_file, usecols=["lemmatized_tweet_token"], lineterminator='\n')
    preprocessed =[]
    token_list = []
    counter = 0
    
    for ind in df.index:
        token_list += [ast.literal_eval(df['lemmatized_tweet_token'][ind])]

    return token_list

In [17]:
all_tokens = get_data_frame(4 * 10 ** 6, is_chunked = True)

In [18]:
all_dictionary = get_dictionary(all_tokens)

In [19]:
all_corpus = get_bow_corpus(all_dictionary, all_tokens)

In [20]:
vocab_length = len(all_dictionary)

best_gsdm_model = {'topics': 6, 'iterations': 10, 'alpha': 0.10, 'beta': 0.01}
gsdmm = MovieGroupProcess(K=6, alpha=0.10, beta=0.01, n_iters=15)
y = gsdmm.fit(tqdm(all_tokens), vocab_length)
doc_count = np.array(gsdmm.cluster_doc_count)

top_index = doc_count.argsort()[-15:][::-1]

100%|██████████████████████████████████████████████████████████████████████████| 45359/45359 [00:00<00:00, 72577.53it/s]


In stage 0: transferred 33526 clusters with 6 clusters populated
In stage 1: transferred 23697 clusters with 6 clusters populated
In stage 2: transferred 16432 clusters with 6 clusters populated
In stage 3: transferred 11683 clusters with 6 clusters populated
In stage 4: transferred 9921 clusters with 6 clusters populated
In stage 5: transferred 9142 clusters with 6 clusters populated
In stage 6: transferred 8566 clusters with 6 clusters populated
In stage 7: transferred 8336 clusters with 6 clusters populated
In stage 8: transferred 8020 clusters with 6 clusters populated
In stage 9: transferred 7997 clusters with 6 clusters populated
In stage 10: transferred 7860 clusters with 6 clusters populated
In stage 11: transferred 7614 clusters with 6 clusters populated
In stage 12: transferred 7621 clusters with 6 clusters populated
In stage 13: transferred 7555 clusters with 6 clusters populated
In stage 14: transferred 7385 clusters with 6 clusters populated


In [21]:
for words in [3, 6]:
    topics = get_topics_lists(gsdmm, top_index, words, all_dictionary)

    cm_gsdmm = CoherenceModel(topics=topics, 
                  dictionary=all_dictionary, 
                  corpus=all_corpus, 
                  texts=all_tokens, 
                  coherence='c_v')
    coherence = cm_gsdmm.get_coherence()
    print("coherence {}: {}".format(words, coherence))

coherence 3: 0.7580767987633772
coherence 6: 0.5820676868732743


In [22]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts = sorted(cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster, sort_dicts))

# get top words in topics
top_words(gsdmm.cluster_word_distribution, top_index, 20)


Cluster 3 : [('دورکاری', 11815), ('کار', 5987), ('خونه', 2891), ('دورکار', 2139), ('شرکت', 1300), ('کرونا', 1112), ('کاشت', 1024), ('صبح', 1011), ('زد', 920), ('میشه', 874), ('😂', 725), ('سرکار', 689), ('مرد', 662), ('خانه', 645), ('بد', 597), ('شب', 576), ('انجام', 552), ('قرنطینه', 518), ('منزل', 501), ('بچه', 500)]

Cluster 0 : [('دورکاری', 8706), ('کار', 6465), ('شرکت', 1910), ('دورکار', 1796), ('خونه', 1549), ('کرونا', 1238), ('میشه', 1157), ('کاشت', 1088), ('ریموت', 931), ('انجام', 843), ('حضور', 832), ('حقوق', 722), ('خانه', 683), ('منزل', 630), ('بشه', 623), ('کنن', 612), ('شرایط', 609), ('ایران', 563), ('تعطیل', 559), ('کارمند', 532)]

Cluster 4 : [('دورکاری', 5418), ('کار', 4321), ('شرکت', 1039), ('منزل', 904), ('کرونا', 846), ('دورکار', 825), ('خانه', 748), ('ریموت', 543), ('اینترنت', 514), ('میشه', 489), ('شرایط', 486), ('کاشت', 481), ('ایران', 446), ('انجام', 424), ('شغل', 389), ('صورت', 372), ('حضور', 338), ('استفاده', 334), ('کسب', 332), ('نیرو', 330)]

Cluster 1 : [('د

## 